In [1]:
import os
import pickle
import sys
from argparse import ArgumentParser

import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.models as models
import tqdm
from torch.nn import functional as fnn
from torch.utils import data
from torchvision import transforms

from hack_utils import NUM_PTS, CROP_SIZE
from hack_utils import ScaleMinSideToSize, CropCenter, TransformByKeys
from hack_utils import ThousandLandmarksDataset
from hack_utils import restore_landmarks_batch, create_submission

torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False

# Составляем аргументы

In [2]:
class myargs():
    pass
args = myargs()

args.batch_size = 128
args.data = 'data'

In [3]:
# 1. prepare data & models
train_transforms = transforms.Compose([
    ScaleMinSideToSize((CROP_SIZE, CROP_SIZE)),
    CropCenter(CROP_SIZE),
    TransformByKeys(transforms.ToPILImage(), ("image",)),
    TransformByKeys(transforms.ToTensor(), ("image",)),
    TransformByKeys(transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5]), ("image",)),
    ])

# Чтение данных

In [4]:
print("Reading data...")
train_dataset = ThousandLandmarksDataset(os.path.join(args.data, 'train'), train_transforms, split="train")
train_dataloader = data.DataLoader(train_dataset, batch_size=args.batch_size, num_workers=4, pin_memory=True,
                                       shuffle=True, drop_last=True)
val_dataset = ThousandLandmarksDataset(os.path.join(args.data, 'train'), train_transforms, split="val")
val_dataloader = data.DataLoader(val_dataset, batch_size=args.batch_size, num_workers=4, pin_memory=True,
                                     shuffle=False, drop_last=False)

Reading data...


315115it [11:43, 720.35it/s]
0it [00:00, ?it/s]
20537it [00:00, 205369.71it/s]
40528it [00:00, 203698.27it/s]
59433it [00:00, 199070.76it/s]
80009it [00:00, 201030.21it/s]
100558it [00:00, 202346.03it/s]
121181it [00:00, 203493.39it/s]
141892it [00:00, 204562.64it/s]
162656it [00:00, 205473.23it/s]
183260it [00:00, 205641.82it/s]
203724it [00:01, 205339.57it/s]
224625it [00:01, 206425.84it/s]
245325it [00:01, 206596.33it/s]
265704it [00:01, 199018.23it/s]
286542it [00:01, 201735.89it/s]
307217it [00:01, 203213.31it/s]
322468it [00:11, 4924.10it/s]  
322468it [00:11, 4924.10it/s]
322619it [00:11, 1760.47it/s]
322746it [00:12, 1216.99it/s]
322856it [00:12, 1003.06it/s]
322955it [00:12, 893.16it/s] 
323046it [00:12, 827.08it/s]
323131it [00:12, 788.20it/s]
323212it [00:12, 761.27it/s]
323290it [00:12, 745.29it/s]
323366it [00:12, 734.74it/s]
323441it [00:13, 725.87it/s]
323515it [00:13, 720.23it/s]
323588it [00:13, 716.93it/s]
323660it [00:13, 711.16it/s]
323732it [00:13, 710.42it/s]
3238

# Постреоние модели

In [5]:
args.gpu = True
device = torch.device("cuda: 0") if args.gpu else torch.device("cpu")

In [6]:
model = models.resnet50(pretrained=True)

In [7]:
model.fc = nn.Sequential(
                nn.Dropout(p=0.2),
                nn.Linear(model.fc.in_features, 10000),
                
                nn.Dropout(p=0.2),
                nn.Linear(10000, 7768),
                
                nn.Dropout(p=0.2),
                nn.Linear(7768, 2*NUM_PTS)
            )

In [9]:
#model.to(device)
#print(1)

In [10]:
from torch.optim.lr_scheduler import ReduceLROnPlateau
from torch.nn.functional import smooth_l1_loss

In [35]:
args.learning_rate = 0.0001

optimizer = optim.Adam(model.parameters(), lr=args.learning_rate, amsgrad=True)
#loss_fn = fnn.smooth_l1_loss
loss_fn = fnn.mse_loss

# Обучение

In [36]:
def train(model, loader, loss_fn, optimizer, device):
    model.train()
    train_loss = []
    for batch in tqdm.tqdm(loader, total=len(loader), desc="training..."):
        images = batch["image"].to(device)  # B x 3 x CROP_SIZE x CROP_SIZE
        landmarks = batch["landmarks"]  # B x (2 * NUM_PTS)

        pred_landmarks = model(images).cpu()  # B x (2 * NUM_PTS)
        loss = loss_fn(pred_landmarks, landmarks, reduction="mean")
        train_loss.append(loss.item())

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    return np.mean(train_loss)

def validate(model, loader, loss_fn, device):
    model.eval()
    val_loss = []
    for batch in tqdm.tqdm(loader, total=len(loader), desc="validation..."):
        images = batch["image"].to(device)
        landmarks = batch["landmarks"]

        with torch.no_grad():
            pred_landmarks = model(images).cpu()
        loss = loss_fn(pred_landmarks, landmarks, reduction="mean")
        val_loss.append(loss.item())

    return np.mean(val_loss)

In [39]:
args.epochs=1
args.name = 'own_model_dropout_4'

# Обучение

In [40]:
print("Ready for training...")
best_val_loss = np.inf
for epoch in range(args.epochs):
    train_loss = train(model, train_dataloader, loss_fn, optimizer, device=device)
    val_loss = validate(model, val_dataloader, loss_fn, device=device)
    print("Epoch #{:2}:\ttrain loss: {:5.2}\tval loss: {:5.2}".format(epoch, train_loss, val_loss))
    if val_loss < best_val_loss:
        best_val_loss = val_loss
        with open(f"{args.name}_best.pth", "wb") as fp:
            torch.save(model.state_dict(), fp)



training...:   0%|          | 0/2462 [00:00<?, ?it/s]

Ready for training...




training...:   0%|          | 1/2462 [00:01<1:04:47,  1.58s/it]

training...:   0%|          | 2/2462 [00:02<51:28,  1.26s/it]  

training...:   0%|          | 3/2462 [00:02<42:20,  1.03s/it]

training...:   0%|          | 4/2462 [00:03<35:48,  1.14it/s]

training...:   0%|          | 5/2462 [00:03<31:19,  1.31it/s]

training...:   0%|          | 6/2462 [00:04<28:07,  1.46it/s]

training...:   0%|          | 7/2462 [00:04<25:55,  1.58it/s]

training...:   0%|          | 8/2462 [00:05<24:21,  1.68it/s]

training...:   0%|          | 9/2462 [00:05<23:17,  1.75it/s]

training...:   0%|          | 10/2462 [00:06<22:30,  1.82it/s]

training...:   0%|          | 11/2462 [00:06<21:59,  1.86it/s]

training...:   0%|          | 12/2462 [00:07<21:36,  1.89it/s]

training...:   1%|          | 13/2462 [00:07<21:22,  1.91it/s]

training...:   1%|          | 14/2462 [00:08<21:10,  1.93it/s]

training...:   1%|          | 15/2462 [00:08<21:03,  1.94it/s]

training...:   1%|          | 16/2462 [00:0

Epoch # 0:	train loss:   2.3	val loss:   2.1


# predict

In [41]:
# 3. predict
test_dataset = ThousandLandmarksDataset(os.path.join(args.data, 'test'), train_transforms, split="test")
test_dataloader = data.DataLoader(test_dataset, batch_size=args.batch_size, num_workers=4, pin_memory=True,
                                  shuffle=False, drop_last=False)



0it [00:00, ?it/s]

46794it [00:00, 467932.64it/s]

99820it [00:00, 464393.58it/s]


# submission

In [42]:
def predict(model, loader, device):
    model.eval()
    predictions = np.zeros((len(loader.dataset), NUM_PTS, 2))
    for i, batch in enumerate(tqdm.tqdm(loader, total=len(loader), desc="test prediction...")):
        images = batch["image"].to(device)

        with torch.no_grad():
            pred_landmarks = model(images).cpu()
        pred_landmarks = pred_landmarks.numpy().reshape((len(pred_landmarks), NUM_PTS, 2))  # B x NUM_PTS x 2

        fs = batch["scale_coef"].numpy()  # B
        margins_x = batch["crop_margin_x"].numpy()  # B
        margins_y = batch["crop_margin_y"].numpy()  # B
        prediction = restore_landmarks_batch(pred_landmarks, fs, margins_x, margins_y)  # B x NUM_PTS x 2
        predictions[i * loader.batch_size: (i + 1) * loader.batch_size] = prediction

    return predictions

In [43]:
with open(f"{args.name}_best.pth", "rb") as fp:
    best_state_dict = torch.load(fp, map_location="cpu")
    model.load_state_dict(best_state_dict)

test_predictions = predict(model, test_dataloader, device)
with open(f"{args.name}_test_predictions.pkl", "wb") as fp:
    pickle.dump({"image_names": test_dataset.image_names,
                 "landmarks": test_predictions}, fp)

create_submission(args.data, test_predictions, f"{args.name}_submit.csv")



test prediction...:   0%|          | 0/780 [00:00<?, ?it/s]

test prediction...:   0%|          | 1/780 [00:02<27:54,  2.15s/it]

test prediction...:   0%|          | 2/780 [00:02<20:03,  1.55s/it]

test prediction...:   0%|          | 3/780 [00:02<14:32,  1.12s/it]

test prediction...:   1%|          | 4/780 [00:02<10:40,  1.21it/s]

test prediction...:   1%|          | 5/780 [00:03<11:26,  1.13it/s]

test prediction...:   1%|          | 6/780 [00:03<08:30,  1.52it/s]

test prediction...:   1%|          | 7/780 [00:03<06:27,  2.00it/s]

test prediction...:   1%|          | 8/780 [00:03<05:01,  2.56it/s]

test prediction...:   1%|          | 9/780 [00:04<05:43,  2.24it/s]

test prediction...:   1%|▏         | 10/780 [00:04<04:50,  2.65it/s]

test prediction...:   1%|▏         | 11/780 [00:04<03:53,  3.30it/s]

test prediction...:   2%|▏         | 12/780 [00:05<03:45,  3.41it/s]

test prediction...:   2%|▏         | 13/780 [00:06<06:06,  2.09it/s]

test prediction...:   2%|▏         |